# Classificador Automático de Sentimento

O projeto utiliza o teorema de Bayes para classificar sentimentos baseados em tweets sobre algum determinado assunto ou marca, para esse projeto a marca utilizada foi a Brahma. Foi utilizado o Classificador Naive-Bayes codado de forma manual por ser um trabalho realizado no inicio do curso de engenharia de computação, em 2016. O projeto utiliza a biblioteca tweepy para automaticamente baixar os tweets no Twitter.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [4]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. 

In [5]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"


In [2]:
#Dados de autenticação do twitter:

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [14]:
#Produto escolhido:
produto = 'Brahma'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [15]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [16]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 

___
## Montando o Classificador Naive-Bayes


* Limpando as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigindo separação de espaços entre palavras e/ou emojis.
* Realizando outras limpezas/transformações que não afetem a qualidade da informação.



In [37]:
dados = pd.read_excel('Brahma.xlsx')

In [38]:
relevantes = dados[dados.Classific == 'R'] 
irrelevantes = dados[dados.Classific == 'I']
listaR = [] 
LR = [] 
listaI = [] 
LI = [] 
LT = []
listaT = []
excluir=["-","!","_",";","+","@","/","]",".", ",","=","'","?","%","#","*",":","\,"] 

for i in relevantes.Treinamento: 
    separa=i.split()
    for j in separa:
        letras=list(j)
        for l in letras:
            if l in excluir:
                letras.remove(l)
        palavra=''.join(letras)
        listaR.append(palavra)
        
for i in irrelevantes.Treinamento: 
    separa1=i.split()
    for j in separa1:
        letras1=list(j)
        for l1 in letras1:
            if l1 in excluir:
                letras1.remove(l1)
        palavra1=''.join(letras1)
        listaI.append(palavra1) 
        
for i in dados.Treinamento: 
    separa2=i.split()
    for j in separa2:
        letras2=list(j)
        for l2 in letras2:
            if l2 in excluir:
                letras2.remove(l2)
        palavra2=''.join(letras2)
        listaT.append(palavra2)    


for h in listaR: 
    if h not in LR:
        LR.append(h)

for h in listaI:  
    if h not in LI:
        LI.append(h)

for h in listaT:  
    if h not in LT:
        LT.append(h)
        
        
LT=list(set(LT))
NumR = len(LR)
NumI= len(LI)

___
## Verificando a performance


* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)



In [39]:
dadosteste=pd.read_excel("Brahma.xlsx",sheetname="Teste")

PF=0
PV=0
NF=0
NV=0
muito_relevante=0 
relevante=0
neutro=0
irrelevante=0
muito_irrelevante=0

ProbabilidadeR=[]
Relevantes=[1]
ProbabilidadeI=[]
Irrelevantes=[1]
ListaRe=[]
ListaIR=[]

In [40]:
for i in range(len(dadosteste.Teste)): 
    ProbabilidadeR.append([]) 
    ProbabilidadeI.append([]) 
    for j in range(len(dadosteste.Teste[i].split())): 
        conta1=LR.count(dadosteste.Teste[i].split()[j]) 
        conta2=LI.count(dadosteste.Teste[i].split()[j])
        
        ProbabilidadeR[i].append((conta1+1)/(NumR+len(LT))) 
        ProbabilidadeI[i].append((conta2+1)/(NumI+len(LT)))      
        
        Relevantes[i]=(Relevantes[i]*ProbabilidadeR[i][j]) 
        Irrelevantes[i]=(Irrelevantes[i]*ProbabilidadeI[i][j])
        
    Relevantes.append(1)
    Irrelevantes.append(1)       
    
for i in range(len(dadosteste["Classific"])):
   
    if Relevantes[i]<Irrelevantes[i]:
        
        ListaRe.append(Relevantes[i])
        if dadosteste.Classific[i]=="R":
            
            PV+=1
        else:
            PF+=1                 
            
    else:
        ListaIR.append(Irrelevantes[i])
        if dadosteste.Classific[i]=="I":            
            NV+=1
            
        else:            
            NF+=1
ListaRe.sort()
ListaIR.sort()

vcentral_relevancia = ListaRe[int(len(ListaRe)/2)]
vcentral_irrelevancia =  ListaIR[int(len(ListaRe)/2)]



for i in range(len(ListaRe)):
    if Relevantes[i]>=ListaRe[int(len(ListaRe)*0.75)]:
        muito_relevante+=1
    else:
        if Relevantes[i]>=ListaRe[int(len(ListaRe)*0.25)]:
            relevante+=1
        else:
            neutro+=1

for i in range(len(ListaIR)):
    if Irrelevantes[i]>=ListaIR[int(len(ListaIR)*0.75)]:
        muito_irrelevante+=1
    else:
        if Irrelevantes[i]>=ListaIR[int(len(ListaIR)*0.25)]:
            irrelevante+=1
        else:
            neutro+=1


probabilidadeFP=PF/dadosteste["Teste"].shape[0]
probabilidadeVP=PV/dadosteste["Teste"].shape[0]
probabilidadeFN=NF/dadosteste["Teste"].shape[0]
probabilidadeVN=NV/dadosteste["Teste"].shape[0]

In [41]:
print("Positivos Verdadeiros :{0:.2f}%".format(probabilidadeVP*100))
print("Negativos Verdadeiros: {0:.2f}%".format(probabilidadeVN*100))
print("Positivos Falsos: {0:.2f}%".format(probabilidadeFP*100))
print("Negativos Falsos : {0:.2f}%".format(probabilidadeFN*100))

Positivos Verdadeiros :1.00%
Negativos Verdadeiros: 69.00%
Positivos Falsos: 2.50%
Negativos Falsos : 27.50%


In [42]:
print("Muito Relevantes :{0:.2f}%".format((muito_relevante/len(dadosteste["Classific"]))*100))
print("Relevantes: {0:.2f}%".format((relevante/len(dadosteste["Classific"]))*100))
print("Neutros: {0:.2f}%".format((neutro/len(dadosteste["Classific"]))*100))
print("Irrelevantes : {0:.2f}%".format((irrelevante/len(dadosteste["Classific"]))*100))   
print("Muito Irrelevantes : {0:.2f}%".format((muito_irrelevante/len(dadosteste["Classific"]))*100))

Muito Relevantes :0.00%
Relevantes: 2.00%
Neutros: 12.50%
Irrelevantes : 60.00%
Muito Irrelevantes : 25.50%


In [44]:
print("Total de Acertos :{0:.2f}%".format(probabilidadeVP*100+probabilidadeVN*100))
print("Total de Erros  : {0:.2f}%".format(probabilidadeFP*100+probabilidadeFN*100))

Total de Acertos :70.00%
Total de Erros  : 30.00%


___
## Concluindo


    
 Para montar o algoritimo nos baseamos no classificador Naive-Bayes que classifica as mensagens em Relevantes ou Irrelevantes seguindo o conceito de machine learning. Analisando palavra por palavra usando o conceito de probabilidade, podemos discorrer que o classificador é preciso, porém não consegue analisar casos de dupla negação, sarcasmo ou alguma forma de exprimir sentimentos por meio de um significado implícito no contexto da frase. No caso do produto analisado por ser uma marca de cerveja, um produto popular, barato e associado a um entorpecimento mental, podemos observar muitos tweets relacionados a promoções, pessoas informando o consumo do produto, sentimentos demonstrados por meio de emoticons, frases inacabadas e opiniões implicitas que dificultam quando a classificação é palavra por palavra. A seguir podemos analisar 5 exemplos de tweets que o método de analisar palavra por palavra não funciona:
 
 *  só uma menina pra beber uma brahma comigo amanhã ao som de art popular...
 *  nunca vi uma terça com tanta cara de sexta, na moral só queria eu e meus amigos num bar tomando um choppzinho da brahma bem gelado
 *  como q alguem bebe brahma meu deus
 *  dia ta pedindo uma praia e umas brahma
 *  brahma é brahma né!? 😍
 
 Percebemos com isso que o classificador acaba perdendo muita qualidade e até se confunde às vezes, entretanto por ser muito maleável seria ideal para analisar outros produtos ou outras redes sociais onde o publico alvo escreva de forma mais clara e opinativa. Segundo a taxa alta (69%) de negativos verdadeiros percebemos que o algoritmo entende bem a não relevância alta dos tweets seguindo a classificação humana, e como podemos enxergar abaixo a grande maioria Neutros, Irrelevantes e Muito Irrelevantes. 
 
- Muito Relevantes :0.00%
- Relevantes: 2.00%
- Neutros: 12.50%
- Irrelevantes : 60.00%
- Muito Irrelevantes : 25.50%

 No caso da empresa Brahma analisando as mensagens no twitter para realmente haver uma plano de expanção e um aumento na eficácia do comparador seria necessario um código mais complexo capaz de analisar frases ao invés de apenas palavras, implementando uma forma de analisar emoticons e até figuras de linguagem como metáforas e onomatopeias por exemplo. Retirando todos os tweets repetidos também. 




